In [36]:
#import stuff
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from pathlib import Path

In [37]:
SCOPES = ["https://www.googleapis.com/auth/script.projects",
          "https://www.googleapis.com/auth/tasks"]
CRED_PATH = Path("secrets/credentials.json")

def get_credentials(cred_path : Path ,
                    token_dest : Path = Path("secrets/token.json")) -> Credentials:

    cred_path = cred_path.resolve()
    token_dest = token_dest.resolve()

    creds = None

    if token_dest.exists():
        creds = Credentials.from_authorized_user_file(token_dest, SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(cred_path, SCOPES)
        creds = flow.run_local_server(port=0)
        with open(token_dest,"w") as token:
            token.write(creds.to_json())
    
    return creds

creds = get_credentials(CRED_PATH)


In [38]:
def start_service():
    service = build("tasks", "v1", credentials = creds)
    return service

serv = start_service()

In [39]:
results = serv.tasklists().list(maxResults=10).execute()
items = results.get("items", [])
items


[{'kind': 'tasks#taskList',
  'id': 'MDI1NDI3OTg2MjMzNjU1OTg3NTc6MDow',
  'etag': '"OTQ5NTE2OTk2"',
  'title': 'My Tasks',
  'updated': '2025-02-24T15:34:31.984Z',
  'selfLink': 'https://www.googleapis.com/tasks/v1/users/@me/lists/MDI1NDI3OTg2MjMzNjU1OTg3NTc6MDow'},
 {'kind': 'tasks#taskList',
  'id': 'dHZNa3ZHcDczc2NWbGtGUw',
  'etag': '"LTEwMjcxMjkzMTM"',
  'title': 'Long term',
  'updated': '2025-02-01T18:30:25.940Z',
  'selfLink': 'https://www.googleapis.com/tasks/v1/users/@me/lists/dHZNa3ZHcDczc2NWbGtGUw'},
 {'kind': 'tasks#taskList',
  'id': 'bjlMUkZ4bFBpdm1YekdmUg',
  'etag': '"LTE4MzQ2ODg0MDM"',
  'title': 'Today',
  'updated': '2023-01-08T18:29:17.387Z',
  'selfLink': 'https://www.googleapis.com/tasks/v1/users/@me/lists/bjlMUkZ4bFBpdm1YekdmUg'}]

In [46]:
results_tasklist = serv.tasks().list(tasklist="bjlMUkZ4bFBpdm1YekdmUg").execute()
results_tasklist

{'kind': 'tasks#tasks',
 'etag': '"LTE4MzQ2ODg0MDM"',
 'items': [{'kind': 'tasks#task',
   'id': 'dGZhLVY1Um1RV2ZMTmE4Vw',
   'etag': '"MjAzODcwNjIyOA"',
   'title': 'Password reset for ARIA',
   'updated': '2023-01-03T21:23:04.000Z',
   'selfLink': 'https://www.googleapis.com/tasks/v1/lists/bjlMUkZ4bFBpdm1YekdmUg/tasks/dGZhLVY1Um1RV2ZMTmE4Vw',
   'position': '00000000000000000000',
   'status': 'needsAction',
   'links': [],
   'webViewLink': 'https://tasks.google.com/task/tfa-V5RmQWfLNa8W?sa=6'}]}

In [ ]:
task_body = "{}"
results_insert = serv.tasks().insert(tasklist ="bjlMUkZ4bFBpdm1YekdmUg")